In [85]:
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,lpDot


In [56]:
data = pd.read_csv("matriz.csv")
data

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,1,46,49,9,17,24,49,38,31,34,0
1,2,18,24,28,2,21,36,30,38,9,0
2,3,27,21,30,40,47,25,17,10,39,0
3,4,28,20,29,20,48,30,32,1,36,32
4,5,5,33,3,44,36,24,50,16,4,45
5,6,5,6,21,8,16,39,29,7,16,17
6,7,31,22,6,42,20,38,50,40,32,42
7,8,1,28,37,19,37,19,23,6,34,31
8,9,29,48,6,43,47,9,42,28,26,21
9,10,20,9,18,35,37,13,34,4,44,16


In [57]:
data.iloc[:,[1,2,3]] # .iloc acessa as posições do df
data.iloc[:,1:3]
data.iloc[:,:]

new_df = data.iloc[:,1:]
 
data.iloc[:,0]

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: Unnamed: 0, dtype: int64

In [75]:
print(new_df)


   X1  X2  X3  X4  X5  X6  X7  X8  X9  X10
0  46  49   9  17  24  49  38  31  34    0
1  18  24  28   2  21  36  30  38   9    0
2  27  21  30  40  47  25  17  10  39    0
3  28  20  29  20  48  30  32   1  36   32
4   5  33   3  44  36  24  50  16   4   45
5   5   6  21   8  16  39  29   7  16   17
6  31  22   6  42  20  38  50  40  32   42
7   1  28  37  19  37  19  23   6  34   31
8  29  48   6  43  47   9  42  28  26   21
9  20   9  18  35  37  13  34   4  44   16


In [105]:
matriz = new_df.values
matriz
matriz[0:3,:]


numpy.ndarray

In [60]:
""" 
Estamos considerando a ultima coluna como o valor das restrições.

a função objetiva a ser otimizada no e-restrito deve ser a equação da primeira linha
as outra p funções devem ser passadas


"""


' \nEstamos considerando a ultima coluna como o valor das restrições\n\n\n\n'

In [111]:
def e_restrito_csv_to_numpy(df,num_funcoes,epsilons):
    
    matriz = df.values
    # Número de variáveis
    n = np.shape(df)[1]-1
    m = np.shape(df)[1]-1
    # Número de funções (num_funcoes)
    # matriz com as funções
    func_principal = matriz[0,:m] 
    matriz_func = matriz[1:num_funcoes,:m]
    # Restrições
    restricao = matriz[num_funcoes:,n]
    matriz_res = matriz[num_funcoes:,0:n]
    #return(matriz_res)

    model = LpProblem(name="large-problem", sense=LpMaximize)
   # Neste trabalho será tratado apenas dos casos lineares com variáveis reais

    x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(n)]
    # Restrições <=
    if isinstance(matriz_res, (list, np.ndarray)) and isinstance(restricao, (list, np.ndarray)):
        rest1 = matriz_res.shape[0]
    
    # Criamos Uma lista que contêm todas as variáveis do problema.

        for i in range(rest1):
            model += (lpDot(matriz_res[i], x) == restricao[i], f"restrição {i}")
    else:
        pass
    

    funcao_principal = lpDot(func_principal,x)
    model += funcao_principal


    for i in range(num_funcoes-1):
        model += (lpDot(matriz_func[i], x) == epsilons[i], f"restrição {rest1+i}")


    status = model.solve()
    sol = np.zeros(n)
    for i in range(n):
        sol[i] = x[i].value()
    print(f"Solução: {sol}")
    print(type(sol))
    print(type(matriz_func))
    sol = np.transpose(sol)
    value_func_aux = np.zeros(num_funcoes-1)
    
    for i in range(num_funcoes-1):
     value_func_aux[i] = np.dot(matriz_func[i],sol)
    for i in range(num_funcoes-1):
        print(f"O valor de f{i+2} é: {value_func_aux[i]}")
    print(f"status: {model.status}, {LpStatus[model.status]}")
    print(f"Objetivo: {model.objective.value()}")

    

e_restrito_csv_to_numpy(new_df,3,np.array([2,3]))















Solução: [ 0.          0.          0.          0.          0.13314895 -0.43998702
  1.0002117   0.          0.18893577]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
O valor de f2 é: 18.663368159999997
O valor de f3 é: 19.63041908
status: -1, Infeasible
Objetivo: 26.068071599999996


In [114]:


# Coeficientes fornecidos
coeficientes = [
    [1, 1, 0, 0, 0, 0],
    [1, -1, 0, 0, -1, 0],
    [2, 1, 1, 0, 0, 100],
    [1, 2, 0, -1, 0, 10]
]

# Escrevendo os coeficientes em um DataFrame do Pandas
df = pd.DataFrame(coeficientes, columns=['X1', 'X2', 'X3', 'X4', 'X5', 'X6'])

# Escrevendo o DataFrame em um arquivo CSV
df.to_csv('coeficientes.csv', index=False)

print("Arquivo 'coeficientes.csv' criado com sucesso.")

Arquivo 'coeficientes.csv' criado com sucesso.


In [116]:
data2= pd.read_csv("coeficientes.csv")
e_restrito_csv_to_numpy(data2,2,np.array([50]))



Solução: [50.  0.  0. 40.  0.]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
O valor de f2 é: 50.0
status: 1, Optimal
Objetivo: 50.0
